In [1]:
import mne
from mne.io import concatenate_raws, read_raw_edf
import matplotlib.pyplot as plt
import mne.viz
import os
import os.path as op
from mne.preprocessing import (ICA, create_eog_epochs, create_ecg_epochs,
                               corrmap)
from mne.time_frequency import tfr_multitaper
from mne.stats import permutation_cluster_1samp_test as pcluster_test
from mne.datasets import fetch_fsaverage

import torch
from torcheeg.models import EEGNet
from torcheeg.models import DGCNN
from torcheeg import transforms

import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as signal
import scipy as sc
from matplotlib import cm
import matplotlib.colors as colors

In [5]:
import sys 
sys.path.append("/home/aurelien.stumpf/Development/BCI_Classification")
from eeg_project_package import dataset, models, training

In [1]:
import importlib
importlib.reload(dataset)
importlib.reload(training)

NameError: name 'dataset' is not defined

In [7]:
#Define the parameters
subject = 1  # use data from subject 1
runs = [6, 10, 14]  # use only hand and feet motor imagery runs

#Get data and locate in to given path
input_fname1 =  "/network/lustre/iss02/aramis/projects/cracknetworks/sub-14/ses-03/EEG/Train-[2022.05.05-15.00.03].edf"
input_fname2 =  "/network/lustre/iss02/aramis/projects/cracknetworks/sub-14/ses-03/EEG/Train-[2022.05.05-15.09.24].edf"
input_fname3 =  "/network/lustre/iss02/aramis/projects/cracknetworks/sub-14/ses-03/EEG/Train-[2022.05.05-15.18.26].edf"
#Read raw data files where each file contains a run
raws = read_raw_edf(input_fname1, preload=True)
#Combine all loaded runs
#raw_obj = concatenate_raws(raws)
raw_obj = raws

raw_data = raw_obj.get_data()
print("Number of channels: ", str(len(raw_data)))
print("Number of samples: ", str(len(raw_data)))

#Plot epochs & PSD
raw_obj.plot(duration=120, n_channels=15, scalings=dict(eeg=420e-6))
raw_obj.plot_psd(average=True)

FileNotFoundError: [Errno 2] No such file or directory: '/network/lustre/iss02/aramis/projects/cracknetworks/sub-14/ses-03/EEG/Train-[2022.05.05-15.00.03].edf'

In [4]:
total_events, dict = mne.events_from_annotations(raw_obj)

## Time Analysis

In [19]:
list_name_channels = ["CP1","CP3","CP5","C1","C3","C5","C2","CP2","Cz","FCz","C4","CP4"]
ch_names = raw_obj.info['ch_names']
list_idx_channels = [ch_names.index(name) for name in list_name_channels]
#list_idx_channels = list(range(0, 64))

task_type = "imaginary_left/right"
folder_path = "/network/lustre/iss02/aramis/projects/cracknetworks/"

num_subjects = [14]
num_train_sessions = [1,2]
num_test_sessions = [3]
num_runs = []
list_labels = [5,6]

torch_trainset = dataset.EEG_Time_Dataset_Multi_Subject(folder_path, num_subjects, num_train_sessions, num_runs, list_idx_channels, list_labels, data="bci")
torch_testset = dataset.EEG_Time_Dataset_Multi_Subject(folder_path, num_subjects, num_test_sessions, num_runs, list_idx_channels, list_labels, data="bci")
trainloader = torch.utils.data.DataLoader(torch_trainset, batch_size=16, shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(torch_testset, batch_size=16, shuffle=True, num_workers=2)

/home/aurelien.stumpf/Code/eeg_project_package/dataset.py:8: RuntimeWarning: Invalid patient information 
  raw_Training_EEG =  mne.io.read_raw_edf(filepath, preload=True, verbose=False)
/home/aurelien.stumpf/Code/eeg_project_package/dataset.py:8: RuntimeWarning: Invalid patient information 
  raw_Training_EEG =  mne.io.read_raw_edf(filepath, preload=True, verbose=False)
/home/aurelien.stumpf/Code/eeg_project_package/dataset.py:8: RuntimeWarning: Invalid patient information 
  raw_Training_EEG =  mne.io.read_raw_edf(filepath, preload=True, verbose=False)
/home/aurelien.stumpf/Code/eeg_project_package/dataset.py:8: RuntimeWarning: Invalid patient information 
  raw_Training_EEG =  mne.io.read_raw_edf(filepath, preload=True, verbose=False)
/home/aurelien.stumpf/Code/eeg_project_package/dataset.py:8: RuntimeWarning: Invalid patient information 
  raw_Training_EEG =  mne.io.read_raw_edf(filepath, preload=True, verbose=False)
/home/aurelien.stumpf/Code/eeg_project_package/dataset.py:8: Runt

Error in file:  /network/lustre/iss02/aramis/projects/cracknetworks/sub-14/ses-02/EEG/Test-[2022.04.28-16.18.42].edf
[Errno 5] Input/output error: '/network/lustre/iss02/aramis/projects/cracknetworks/sub-14/ses-02/EEG/Test-[2022.04.28-16.18.42].edf'


/home/aurelien.stumpf/Code/eeg_project_package/dataset.py:8: RuntimeWarning: Invalid patient information 
  raw_Training_EEG =  mne.io.read_raw_edf(filepath, preload=True, verbose=False)
/home/aurelien.stumpf/Code/eeg_project_package/dataset.py:8: RuntimeWarning: Invalid patient information 
  raw_Training_EEG =  mne.io.read_raw_edf(filepath, preload=True, verbose=False)
/home/aurelien.stumpf/Code/eeg_project_package/dataset.py:8: RuntimeWarning: Invalid patient information 
  raw_Training_EEG =  mne.io.read_raw_edf(filepath, preload=True, verbose=False)
/home/aurelien.stumpf/Code/eeg_project_package/dataset.py:8: RuntimeWarning: Invalid patient information 
  raw_Training_EEG =  mne.io.read_raw_edf(filepath, preload=True, verbose=False)
/home/aurelien.stumpf/Code/eeg_project_package/dataset.py:8: RuntimeWarning: Invalid patient information 
  raw_Training_EEG =  mne.io.read_raw_edf(filepath, preload=True, verbose=False)
/home/aurelien.stumpf/Code/eeg_project_package/dataset.py:8: Runt

In [23]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
criterion = torch.nn.CrossEntropyLoss()
model = EEGNet(chunk_size=2001,
               num_electrodes=12,
               dropout=0,
               kernel_1=64,
               kernel_2=16,
               F1=8,
               F2=16,
               D=2,
               num_classes=2)
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.MultiplicativeLR(optimizer, lr_lambda=lambda epoch: 0.93)
training.train_model(model, trainloader, testloader, device, criterion, 100, optimizer, scheduler)

Epoch 0: 100%|██████████| 2/2 [00:00<00:00, 11.87batch/s, accuracy=0.533, loss=0.0808]

Epoch 0
lr:  0.009300000000000001
-------------------------


Train set: Avg. loss: 0.0830, Balanced Accuracy: 0.03333333507180214 (3%)
Test set: Avg. loss: 0.1229, Balanced Accuracy: 0.02083333395421505 (2%)
-------------------------


Epoch 10: 100%|██████████| 2/2 [00:00<00:00, 13.11batch/s, accuracy=0.933, loss=0.0106]


Epoch 10
lr:  0.004501035456767429
-------------------------
Train set: Avg. loss: 0.0093, Balanced Accuracy: 0.08055555820465088 (8%)
Test set: Avg. loss: 0.1336, Balanced Accuracy: 0.0260416679084301 (3%)
-------------------------


...

Train set: Avg. loss: 0.0001, Balanced Accuracy: 0.0833333358168602 (8%)
Test set: Avg. loss: 0.2357, Balanced Accuracy: 0.0364583320915699 (4%)
-------------------------


Epoch 90: 100%|██████████| 2/2 [00:00<00:00, 12.26batch/s, accuracy=1, loss=0.000107]

Epoch 90
lr:  1.3550281526944739e-05
-------------------------


Train set: Avg. loss: 0.0001, Balanced Accuracy: 0.0833333358168602 (8%)
Test set: Avg. loss: 0.2359, Balanced Accuracy: 0.0364583320915699 (4%)
-------------------------


Epoch 99: 100%|██████████| 2/2 [00:00<00:00, 13.07batch/s, accuracy=1, loss=0.000104]


## Graph Analysis

In [88]:
list_name_channels = ["CP1","CP3","CP5","C1","C3","C5","C2","CP2","Cz","FCz","C4","CP4"]
ch_names = raw_obj.info['ch_names']
list_idx_channels = [ch_names.index(name) for name in list_name_channels]
#list_idx_channels = list(range(0, 64))

task_type = "imaginary_left/right"
folder_path = "/network/lustre/iss02/aramis/projects/cracknetworks/"

num_subjects = [14]
num_train_sessions = [1,2]
num_test_sessions = [3]
num_runs = []
list_labels = [5,6]

torch_trainset = dataset.EEG_PSD_Dataset_Multi_Subject(folder_path, num_subjects, num_train_sessions, num_runs, list_idx_channels, list_labels, data="bci", band=True, type_="torch")
torch_testset = dataset.EEG_PSD_Dataset_Multi_Subject(folder_path, num_subjects, num_test_sessions, num_runs, list_idx_channels, list_labels, data="bci", band=True, type_="torch")
trainloader = torch.utils.data.DataLoader(torch_trainset, batch_size=16, shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(torch_testset, batch_size=16, shuffle=True, num_workers=2)

Error in file:  /network/lustre/iss02/aramis/projects/cracknetworks/sub-14/ses-02/EEG/Test-[2022.04.28-16.18.42].edf


In [102]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
criterion = torch.nn.CrossEntropyLoss()
model = DGCNN(in_channels=5, num_electrodes=12, hid_channels=24, num_layers=3, num_classes=2)
model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.MultiplicativeLR(optimizer, lr_lambda=lambda epoch: 0.98)
training.train_model(model, trainloader, testloader, device, criterion, 100, optimizer, scheduler)

Epoch 0:   0%|          | 0/18 [00:00<?, ?batch/s]


ValueError: expected 2D or 3D input (got 4D input)